# $\psi_4$

### Authors: Zach Etienne & Patrick Nelson



<a id='initializenrpy'></a>

# Step 1: Initialize core NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1.a: import all needed modules from NRPy+:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm

# Step 1.b: Set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")

# Step 1.c: Given the chosen coordinate system, set up 
#           corresponding reference metric and needed
#           reference metric quantities
# The following function call sets up the reference metric
#    and related quantities, including rescaling matrices ReDD,
#    ReU, and hatted quantities.
rfm.reference_metric()

# Step 1.d: Set spatial dimension (must be 3 for BSSN, as BSSN is 
#           a 3+1-dimensional decomposition of the general 
#           relativistic field equations)
DIM = 3

# Step 1.e: Import all ADM quantities as written in terms of BSSN quantities
import BSSN.ADM_in_terms_of_BSSN as AB
AB.ADM_in_terms_of_BSSN()

initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX


# Step 2: Constructing the 3-Riemann tensor $R_{ik\ell m}$
Analogously to Christoffel symbols, the Riemann tensor is a measure of the curvature of an $N$-dimensional manifold. Thus the 3-Riemann tensor is not simply a projection of the 4-Riemann tensor (see e.g., Eq. 2.7 of [Campanelli *et al* (1998)](https://arxiv.org/pdf/gr-qc/9803058.pdf) for the relation between 4-Riemann and 3-Riemann), as $N$-dimensional Riemann tensors are meant to define a notion of curvature given only the associated $N$-dimensional metric. 

So, given the ADM 3-metric, the Riemann tensor in arbitrary dimension is given by the 3-dimensional version of Eq. 1.19 in Baumgarte & Shapiro's *Numerical Relativity*. I.e.,

$$
R^i_{jkl} = \partial_k \Gamma^{i}_{jl} - \partial_l \Gamma^{i}_{jk} + \Gamma^i_{mk} \Gamma^m_{jl} - \Gamma^{i}_{ml} \Gamma^{m}_{jk},
$$
where $\Gamma^i_{jk}$ is the Christoffel symbol associated with the 3-metric $\gamma_{ij}$:

$$
\Gamma^l_{ij} = \frac{1}{2} \gamma^{lk} \left(\gamma_{ki,j} + \gamma_{kj,i} - \gamma_{ij,k} \right) 
$$

Notice that this equation for the Riemann tensor is equivalent to the equation given in the Wikipedia article on [Formulas in Riemannian geometry](https://en.wikipedia.org/w/index.php?title=List_of_formulas_in_Riemannian_geometry&oldid=882667524):

$$
R^\ell{}_{ijk}=
\partial_j \Gamma^\ell{}_{ik}-\partial_k\Gamma^\ell{}_{ij}
+\Gamma^\ell{}_{js}\Gamma_{ik}^s-\Gamma^\ell{}_{ks}\Gamma^s{}_{ij},
$$
with the replacements $i\to \ell$, $j\to i$, $k\to j$, $l\to k$, and $s\to m$. Wikipedia also provides a simpler form in terms of second-derivatives of three-metric itself (using the definition of Christoffel symbol), so that we need not define derivatives of the Christoffel symbol:

$$
R_{ik\ell m}=\frac{1}{2}\left(
\gamma_{im,k\ell} 
+ \gamma_{k\ell,im}
- \gamma_{i\ell,km}
- \gamma_{km,i\ell} \right)
+\gamma_{np} \left(
\Gamma^n{}_{k\ell} \Gamma^p{}_{im} - 
\Gamma^n{}_{km} \Gamma^p{}_{i\ell} \right).
$$

First we construct the term on the left:

In [2]:
# Step 2: Construct the (rank-4) Riemann curvature tensor associated with the ADM 3-metric:
RDDDD = ixp.zerorank4()
gammaDDdDD = AB.gammaDDdDD

for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                RDDDD[i][k][l][m] = sp.Rational(1,2) * \
                    (gammaDDdDD[i][m][k][l] + gammaDDdDD[k][l][i][m] - gammaDDdDD[i][l][k][m] - gammaDDdDD[k][m][i][l])

... then we add the term on the right:

In [3]:
gammaDD = AB.gammaDD
GammaUDD = AB.GammaUDD

for i in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for m in range(DIM):
                for n in range(DIM):
                    for p in range(DIM):
                        RDDDD[i][k][l][m] += gammaDD[n][p] * \
                            (GammaUDD[n][k][l]*GammaUDD[p][i][m] - GammaUDD[n][k][m]*GammaUDD[p][i][l])

# Step 3: Constructing the rank-4 tensor in Term 1 of $\psi_4$: $R_{ijkl} + 2 K_{i[k} K_{l]j} $

Following Eq. 5.1 in [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf), the rank-4 tensor in the first term of $\psi_4$ is given by

$$
R_{ijkl} + 2 K_{i[k} K_{l]j} = R_{ijkl} + K_{ik} K_{lj} - K_{il} K_{kj}
$$

In [4]:
# Step 3: Construct the (rank-4) tensor in term 1 of psi_4 (referring to Eq 5.1 in 
#   Baker, Campanelli, Lousto (2001); https://arxiv.org/pdf/gr-qc/0104063.pdf
rank4term1 = ixp.zerorank4()
KDD = AB.KDD

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                rank4term1[i][j][k][l] = RDDDD[i][j][k][l] + KDD[i][k]*KDD[l][j] - KDD[i][l]*KDD[k][j]

# Step 4: Constructing the rank-3 tensor in Term 2 of $\psi_4$: 

Following Eq. 5.1 in [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf), the rank-3 tensor in the second term of $\psi_4$ is given by

$$
-8 \left(K_{j[k,l]} + \Gamma^{p}_{j[k} K_{l]p} \right) = 
$$
First let's construct the first term in this sum: $K_{j[k,l]} = \frac{1}{2} (K_{jk,l} - K_{jl,k})$:

In [5]:
# Step 4: Construct the (rank-3) tensor in term 2 of psi_4 (referring to Eq 5.1 in 
#   Baker, Campanelli, Lousto (2001); https://arxiv.org/pdf/gr-qc/0104063.pdf
rank3term2 = ixp.zerorank3()
KDDdD = AB.KDDdD

for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            rank3term2[j][k][l] = sp.Rational(1,2)*(KDDdD[j][k][l] - KDDdD[j][l][k])

... then we construct the secon term in this sum: $\Gamma^{p}_{j[k} K_{l]p} = \frac{1}{2} (\Gamma^{p}_{jk} K_{lp}-\Gamma^{p}_{jl} K_{kp})$:

In [6]:
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            for p in range(DIM):
                rank3term2[j][k][l] += sp.Rational(1,2)*(GammaUDD[p][j][k]*KDD[l][p] - GammaUDD[p][j][l]*KDD[k][p])

Finally, we multiply the term by $-8$:

In [7]:
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            rank3term2[j][k][l] *= sp.sympify(-8)

# Step 5: Constructing the rank-2 tensor in term 3 of $\psi_4$:

Following Eq. 5.1 in [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf), the rank-2 tensor in the third term of $\psi_4$ is given by

$$
+4 \left(R_{jl} - K_{jp} K^p_l + K K_{jl} \right),
$$
where
\begin{align}
R_{jl} &= R^i_{jil} \\
&= \gamma^{im} R_{ijml} \\
K &= K^i_i \\
&= \gamma^{im} K_{im}
\end{align}

First let's build the first term in parentheses

In [8]:
# Step 5: Construct the (rank-2) tensor in term 3 of psi_4 (referring to Eq 5.1 in 
#   Baker, Campanelli, Lousto (2001); https://arxiv.org/pdf/gr-qc/0104063.pdf
rank2term3 = ixp.zerorank2()
gammaUU = AB.gammaUU

for j in range(DIM):
    for l in range(DIM):
        for i in range(DIM):
            for m in range(DIM):
                rank2term3[j][l] += gammaUU[i][m]*RDDDD[i][j][m][l]

... then we add on the second term in parentheses, where $K^p_l = \gamma^{mp} K_{ml}$

In [9]:
for j in range(DIM):
    for l in range(DIM):
        for m in range(DIM):
            for p in range(DIM):
                rank2term3[j][l] += - KDD[j][p]*gammaUU[p][m]*KDD[m][l]

Finally we add the third term in parentheses, and multiply all terms by $+4$:

In [10]:
for j in range(DIM):
    for l in range(DIM):
        for i in range(DIM):
            for m in range(DIM):
                rank2term3[j][l] += gammaUU[i][m]*KDD[i][m]*KDD[j][l]
for j in range(DIM):
    for l in range(DIM):
        rank2term3[j][l] *= sp.sympify(4)

# Step 6: Constructing $\psi_4$ through contractions of the above terms with an arbitrary tetrad

Eq. 5.1 in [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf) writes $\psi_4$ (which is complex) as the contraction of each of the above terms with products of tetrad vectors:

\begin{align}
\psi_4 &= \left[ {R}_{ijkl}+2K_{i[k}K_{l]j}\right]
{n}^i\bar{m}^j{n}^k\bar{m}^l  \\
& -8\left[ K_{j[k,l]}+{\Gamma }_{j[k}^pK_{l]p}\right]
{n}^{[0}\bar{m}^{j]}{n}^k\bar{m}^l \\
& +4\left[ {R}_{jl}-K_{jp}K_l^p+KK_{jl}\right]
{n}^{[0}\bar{m}^{j]}{n}^{[0}\bar{m}^{l]},
\end{align}
where $\bar{m}^\mu$ is the complex conjugate of $m^\mu$, and $n^\mu$ is real. The third term is given by
\begin{align}
{n}^{[0}\bar{m}^{j]}{n}^{[0}\bar{m}^{l]}
&= \frac{1}{2}({n}^{0}\bar{m}^{j} - {n}^{j}\bar{m}^{0} )\frac{1}{2}({n}^{0}\bar{m}^{l} - {n}^{l}\bar{m}^{0} )\\
&= \frac{1}{4}({n}^{0}\bar{m}^{j} - {n}^{j}\bar{m}^{0} )({n}^{0}\bar{m}^{l} - {n}^{l}\bar{m}^{0} )\\
&= \frac{1}{4}({n}^{0}\bar{m}^{j}{n}^{0}\bar{m}^{l} - {n}^{j}\bar{m}^{0}{n}^{0}\bar{m}^{l} - {n}^{0}\bar{m}^{j}{n}^{l}\bar{m}^{0} +  {n}^{j}\bar{m}^{0}{n}^{l}\bar{m}^{0})
\end{align}

Only $m^\mu$ is complex, so we can separate the real and imaginary parts of $\psi_4$ by hand, defining $M^\mu$ to now be the real part of $\bar{m}^\mu$ and $\mathcal{M}^\mu$ to be the imaginary part. All of the above products are of the form ${n}^\mu\bar{m}^\nu{n}^\eta\bar{m}^\delta$, so let's evalute the real and imaginary parts of this product once, for all such terms:

\begin{align}
{n}^\mu\bar{m}^\nu{n}^\eta\bar{m}^\delta
&= {n}^\mu(M^\nu - i \mathcal{M}^\nu){n}^\eta(M^\delta - i \mathcal{M}^\delta) \\
&= \left({n}^\mu M^\nu {n}^\eta M^\delta -
{n}^\mu \mathcal{M}^\nu {n}^\eta \mathcal{M}^\delta \right)+
i \left(
-{n}^\mu M^\nu {n}^\eta \mathcal{M}^\delta
-{n}^\mu \mathcal{M}^\nu {n}^\eta M^\delta
\right)
\end{align}



In [11]:
mre4U = ixp.declarerank1("mre4U",DIM=4)
mim4U = ixp.declarerank1("mim4U",DIM=4)
n4U   = ixp.declarerank1("n4U"  ,DIM=4)

def tetrad_product__Real_psi4(n,Mre,Mim,  mu,nu,eta,delta):
    return +n[mu]*Mre[nu]*n[eta]*Mre[delta] - n[mu]*Mim[nu]*n[eta]*Mim[delta]

def tetrad_product__Imag_psi4(n,Mre,Mim,  mu,nu,eta,delta):
    return -n[mu]*Mre[nu]*n[eta]*Mim[delta] - n[mu]*Mim[nu]*n[eta]*Mre[delta]


psi4_re = sp.sympify(0)
psi4_im = sp.sympify(0)
# First term:
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                psi4_re += rank4term1[i][j][k][l]*tetrad_product__Real_psi4(n4U,mre4U,mim4U, i+1,j+1,k+1,l+1)
                psi4_im += rank4term1[i][j][k][l]*tetrad_product__Imag_psi4(n4U,mre4U,mim4U, i+1,j+1,k+1,l+1)

# Second term:
for j in range(DIM):
    for k in range(DIM):
        for l in range(DIM):
            psi4_re += rank3term2[j][k][l] * \
                       sp.Rational(1,2)*(+tetrad_product__Real_psi4(n4U,mre4U,mim4U, 0,j+1,k+1,l+1) \
                                         -tetrad_product__Real_psi4(n4U,mre4U,mim4U, j+1,0,k+1,l+1) )
            psi4_im += rank3term2[j][k][l] * \
                       sp.Rational(1,2)*(+tetrad_product__Imag_psi4(n4U,mre4U,mim4U, 0,j+1,k+1,l+1) \
                                         -tetrad_product__Imag_psi4(n4U,mre4U,mim4U, j+1,0,k+1,l+1) )
# Third term:
for j in range(DIM):
    for l in range(DIM):
        psi4_re += rank2term3[j][l] * \
                       (sp.Rational(1,4)*(+tetrad_product__Real_psi4(n4U,mre4U,mim4U, 0,j+1,0,l+1) \
                                          -tetrad_product__Real_psi4(n4U,mre4U,mim4U, j+1,0,0,l+1) \
                                          -tetrad_product__Real_psi4(n4U,mre4U,mim4U, 0,j+1,l+1,0) \
                                          +tetrad_product__Real_psi4(n4U,mre4U,mim4U, j+1,0,l+1,0)))
        psi4_im += rank2term3[j][l] * \
                       (sp.Rational(1,4)*(+tetrad_product__Imag_psi4(n4U,mre4U,mim4U, 0,j+1,0,l+1) \
                                          -tetrad_product__Imag_psi4(n4U,mre4U,mim4U, j+1,0,0,l+1) \
                                          -tetrad_product__Imag_psi4(n4U,mre4U,mim4U, 0,j+1,l+1,0) \
                                          +tetrad_product__Imag_psi4(n4U,mre4U,mim4U, j+1,0,l+1,0)))

# Step 7: The quasi-Kinnersley tetrad

To define the Weyl scalars, first a tetrad must be chosen. Below, for compatibility with the [WeylScal4 diagnostic module](https://bitbucket.org/einsteintoolkit/einsteinanalysis/src/master/WeylScal4/), we implement the quasi-Kinnersley tetrad of [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf).

We begin with the vectors given in eqs. 5.6 and 5.7 of the BCL paper,
\begin{align}
    v_1^a &= [-y,x,0] \\
    v_2^a &= [x,y,z] \\
    v_3^a &= {\rm det}(g)^{1/2} g^{ad} \epsilon_{dbc} v_1^b v_2^c,
\end{align}
and carry out the Gram-Schmidt orthonormalization process. Note that these vectors are initially orthogonal to each other in flat spacetime; one is in the $\phi$ direction, one is in $r$, and the third is the cross product of the first two. The vectors $w_i^a$ are placeholders in the code; the final product of the orthonormalization is the vectors $e_i^a$. So,
\begin{align}
e_1^a &= \frac{v_1^a}{\omega_{11}} \\
e_2^a &= \frac{v_2^a - \omega_{12} e_1^a}{\omega_{22}} \\
e_3^a &= \frac{v_3^a - \omega_{13} e_1^a - \omega_{23} e_2^a}{\omega_{33}}, \\
\end{align}
where $\omega_{ij} = v_i^a v_j^b \bar{\gamma}_{ab}$ needs to be updated between steps (to save resources, we can get away with only calculating components as needed), and uses $e_i^a$ instead of $v_i^a$ if it has been calculated.

Once we have orthogonal, normalized vectors, we can contruct the tetrad itself, again drawing on eqs. 5.6. We can draw on SymPy's built-in tools for complex numbers to build the complex vectors $m^a$ and $(m^*)^a$:
\begin{align}
    l^a &= \frac{1}{\sqrt{2}} e_2^a \\
    n^a &= \frac{-1}{\sqrt{2}} e_2^a \\
    m^a &= \frac{1}{\sqrt{2}} (e_3^a + i e_1^a) \\
    (m^*)^a &= \frac{1}{\sqrt{2}} (e_3^a - i e_1^a)
\end{align}

We will also assume that $n^0 = \frac{1}{\sqrt{2}}$ and that $m^0 = (m^*)^0 = 0$. This last assumption in particular will significantly reduce the terms needed to find $\psi_4$.

In [12]:
# Step 7.a: Define the rank-3 version of the Levi-Civita symbol. Amongst
#         other uses, this is needed for the construction of the approximate 
#         quasi-Kinnersley tetrad.
def define_LeviCivitaSymbol_rank3(DIM=-1):
    if DIM == -1:
        DIM = par.parval_from_str("DIM")

    LeviCivitaSymbol = ixp.zerorank3()

    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                # From https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol :
                LeviCivitaSymbol[i][j][k] = (i - j) * (j - k) * (k - i) / 2
    return LeviCivitaSymbol

In [13]:
# Step 7.b: Initialize WeylScalar parameters
thismodule = __name__
# Current option: Approx_QuasiKinnersley = choice made in Baker, Campanelli, and Lousto. PRD 65, 044001 (2002)
par.initialize_param(par.glb_param("char", thismodule, "TetradChoice", "QuasiKinnersley"))

In [15]:
# Step 4b: Set the coordinate system to Cartesian
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

# Step 4c: Set which tetrad is used; at the moment, only one supported option
TetradChoice = par.parval_from_str("TetradChoice")

if TetradChoice == "QuasiKinnersley":
    # Eqs 5.6 in https://arxiv.org/pdf/gr-qc/0104063.pdf
    xmoved = x# - xorig
    ymoved = y# - yorig
    zmoved = z# - zorig

    # Step 5a: Choose 3 orthogonal vectors. Here, we choose one in the azimuthal 
    #          direction, one in the radial direction, and the cross product of the two. 
    # Eqs 5.7
    v1U = ixp.zerorank1()
    v1U[0] = -ymoved
    v1U[1] = xmoved# + offset
    v1U[2] = 0
    v2U = ixp.zerorank1()
    v2U[0] = xmoved# + offset
    v2U[1] = ymoved
    v2U[2] = zmoved
    v3U = ixp.zerorank1()
    LeviCivitaSymbol_rank3 = define_LeviCivitaSymbol_rank3()
    for a in range(DIM):
        for b in range(DIM):
            for c in range(DIM):
                for d in range(DIM):
                    v3U[a] += sp.sqrt(AB.detgamma) * AB.gammaUU[a][d] \
                                * LeviCivitaSymbol_rank3[d][b][c] * v1U[b] *v2U[c]

    # Step 5b: Gram-Schmidt orthonormalization of the vectors.
    # The w_i^a vectors here are used to temporarily hold values on the way to the final vectors e_i^a

    w1U = ixp.zerorank1()
    for a in range(DIM):
        w1U[a] = v1U[a]
    omega11 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega11 += w1U[a] * w1U[b] * AB.gammaDD[a][b]
    e1U = ixp.zerorank1()
    for a in range(DIM):
        e1U[a] = w1U[a] / sp.sqrt(omega11)

    omega12 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega12 += e1U[a] * v2U[b] * AB.gammaDD[a][b]
    w2U = ixp.zerorank1()
    for a in range(DIM):
        w2U[a] = v2U[a] - omega12*e1U[a]
    omega22 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega22 += w2U[a] * w2U[b] *AB.gammaDD[a][b]
    e2U = ixp.zerorank1()
    for a in range(DIM):
        e2U[a] = w2U[a] / sp.sqrt(omega22)

    omega13 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega13 += e1U[a] * v3U[b] * AB.gammaDD[a][b]
    omega23 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega23 += e2U[a] * v3U[b] * AB.gammaDD[a][b]
    w3U = ixp.zerorank1()
    for a in range(DIM):
        w3U[a] = v3U[a] - omega13*e1U[a] - omega23*e2U[a]
    omega33 = 0
    for a in range(DIM):
        for b in range(DIM):
            omega33 += w3U[a] * w3U[b] * AB.gammaDD[a][b]
    e3U = ixp.zerorank1()
    for a in range(DIM):
        e3U[a] = w3U[a] / sp.sqrt(omega33)
        
    # Step 5c: Construct the tetrad itself.
    # Eqs. 5.6
    isqrt2 = 1/sp.sqrt(2)
    ltetU = ixp.zerorank1()
    ntetU = ixp.zerorank1()
    mtetU = ixp.zerorank1()
    mtetccU = ixp.zerorank1()
    for i in range(DIM):
        ltetU[i] = isqrt2 * e2U[i]
        ntetU[i] = -isqrt2 * e2U[i]
        mtetU[i] = isqrt2 * (e3U[i] + sp.I*e1U[i])
        mtetccU[i] = isqrt2 * (e3U[i] - sp.I*e1U[i])
    nn = isqrt2

else:
    print("Error: TetradChoice == "+par.parval_from_str("TetradChoice")+" unsupported!")
    exit(1)